In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

### Базовый бутстрап

In [104]:
def t_procentile_bootstrap_test(c:list, t:list, quantile:float, B:int, alpha:float, orientation:str) -> (bool, float):
    """
    Функция для проверки гипотезы о равенстве статистики бутстрапируя t-статистику.
    Используется перцентильный доверительный интервал.
    
    Возвращает булево значение о равенстве нулевой гипотезы и p_value.
    -------------------------------------------------------
    c - контрольная выборка
    t - тритмент выборка
    quantile - квантиль, гипотеза о равенстве которого тестируется
    B - количество итераций бутсрапирования
    alpha - уровень значимости
    orientation - направление теста
    """
    # проверка того, что в функцию передано корректное значение направления теста
    if orientation not in ('left', 'right', 'two_sided'):
        raise ValueError('Unknown orientation type')
    
    c_theta = np.quantile(c, quantile)
    t_theta = np.quantile(t, quantile)
    
    # параметр для рецентрирования
    z = np.concatenate([c, t])
    z_theta = np.quantile(z, quantile)
    
    # рецентрирование исходных выборок отностиельно выбранной статистики
    c_new = c - c_theta + z_theta
    t_new = t - t_theta + z_theta
    
    # бутстрапирование t-статистики
    t_st_values = [ ]
    for _ in range(B):
        c_st = np.random.choice(c_new, len(c_new), replace=True)
        t_st = np.random.choice(t_new, len(t_new), replace=True)

        c_theta_st = np.quantile(c_st, quantile)
        t_theta_st = np.quantile(t_st, quantile)

        c_var_st = np.var(c_st)
        t_var_st = np.var(t_st)

        # расчет бустрапированной t-статистики
        t_st = (t_theta_st - c_theta_st) / np.sqrt(c_var_st / len(c_st) + t_var_st / len(t_st))
        t_st_values.append(t_st)      
        
    # расчет наблюдаемого значения t-статистики
    c_var = np.var(c)
    t_var = np.var(t)
    t_obs = (t_theta - c_theta) / np.sqrt(c_var / len(c) + t_var / len(t))
    
    # расчет критических значений t-статистики и проверка гипотезы
    if orientation == 'right':
        t_crit = np.quantile(t_st_values, 1-alpha)
        accept = True if t_obs <= t_crit else False
        p_value = sum(t_st_values >= t_obs) / len(t_st_values)
    elif orientation == 'left':
        t_crit = np.quantile(t_st_values, alpha)
        accept = True if t_obs >= t_crit else False
        p_value = sum(t_st_values <= t_obs) / len(t_st_values)
    else:                                                      # 2х сторонняя ориентация эксперимента
        t_crit_left = np.quantile(t_st_values, alpha/2)
        t_crit_right = np.quantile(t_st_values, 1-alpha/2)
        accept = True if t_crit_left <= t_obs <= t_crit_right else False
        p_value = sum(t_st_values <= t_obs) / len(t_st_values) if t_obs < 0 else sum(t_st_values >= t_obs) / len(t_st_values)
    
    return accept, p_value

### Пуассоновский бутстрап

In [125]:
def poisson_bootstrap(sample:list, quantile:float, B:int) -> (np.array, np.array, np.array):
    """
    Функция для генерации бутстрапированного распределения theta по данным из исходного расперделения.
    Для генерации используется Пуассоновское распределение, 
    являющееся апроксимацией придлеьного случая биномиального распределения при n, стремящемся к бесконечности
    
    Возвращает массив с бутсрапированной статисткиой, 
    массив с дисперсией бутсрапированных выборок,
    массив с размером бутстрапированных выборок
    -------------------------------------------------------
    с - исходная выборка
    quantile - статистика, для которой ищем распределение
    B - количество итераций бутстрапирования
    
    """
    result = {} # словарь, в который будем записывать как часто встречается значение в итерации
    
    # запускаем цикл по исходной выборке, чтобы определеть частоту для каждого элемента
    for el in c:
        # генерируем частоту встречания элемента в бутсрапированных выборках
        values = np.random.poisson(1, B)
        
        # записываем полученные часоты в словарь. Если елемент встречается несколько раз, то частоты суммируются
        result[el] = result.setdefault(el, 0) + values        

    # массив, в который будут записаны значения бутстрапированных статистик
    thetas_st = [ ]
    # массив, в который будут передаваться дисперсии бутстрапированных выборок
    sample_st_vars = [ ]
    # массив, в который будут передаваться размеры бутстрапированных выборок
    sample_st_lens = [ ]
    for i in range(B):
        sample_st = [ ] # массив со значениями одной i-той бутстрапированной выборки
        for k in result.keys():
            # получение частоты встречания k-го элемена в i-ой выбрке
            obs_num = result[k][i]
            # генерация массива с количеством k-го елемена
            obs = [k] * obs_num
            # запись k-тых элементов в i-ю выборку
            sample_st.extend(obs)

        # расчет искомой бутсрапированной статистики по i-й выборке и ее дисперсии
        theta_st = np.quantile(sample_st, quantile)
        sample_st_var = np.var(sample_st)

        # сохранение бутстрапированной статистики в массив
        thetas_st.append(theta_st)
        # сохранение дисперсии бутстрапированной выборки в массив
        sample_st_vars.append(sample_st_var)
        #сохранение рамера бутстрапированных выборок в массив
        sample_st_lens.append(len(sample_st))
        
    return np.array(thetas_st), np.array(sample_st_vars), np.array(sample_st_lens)


def t_procentile_poisson_bootstrap_test(c:list, t:list, quantile:float,
                                        B:int, alpha:float, orientation:str) -> (bool, float):
    """
    Функция для проверки гипотезы о равенстве статистики бутстрапируя t-статистику.
    Бутстрапирование осуществляется "пуассоновским бутсрапом".
    Используется перцентильный доверительный интервал
    
    Возвращает булево значение о равенстве нулевой гипотезы и p_value.
    -------------------------------------------------------
    c - контрольная выборка
    t - тритмент выборка
    quantile - квантиль, гипотеза о равенстве которого тестируется
    B - количество итераций бутсрапирования
    alpha - уровень значимости
    orientation - направление теста
    """
    # генерация массивов для формирования распределения t-статистики
    c_st_boot, c_var_boot, c_len_boot = poisson_bootstrap(c, quantile, B)
    t_st_boot, t_var_boot, t_len_boot = poisson_bootstrap(t, quantile, B)
    
    # формирование распределения t-статистики
    t_st_values = (t_st_boot - c_st_boot) / np.sqrt(c_var_boot / c_len_boot + t_var_boot / t_len_boot)
    
    # расчет наблюдаемого t-значения
    c_theta = np.quantile(c, quantile)
    t_theta = np.quantile(t, quantile)
    c_var = np.var(c)
    t_var = np.var(t)
    t_obs = (t_theta - c_theta) / np.sqrt(c_var / len(c) + t_var / len(t))
    
    # расчет критических значений t-статистики и проверка гипотезы
    if orientation == 'right':
        t_crit = np.quantile(t_st_values, 1-alpha)
        accept = True if t_obs <= t_crit else False
        p_value = sum(t_st_values >= t_obs) / len(t_st_values)
    elif orientation == 'left':
        t_crit = np.quantile(t_st_values, alpha)
        accept = True if t_obs >= t_crit else False
        p_value = sum(t_st_values <= t_obs) / len(t_st_values)
    else:                                                      # 2х сторонняя ориентация эксперимента
        t_crit_left = np.quantile(t_st_values, alpha/2)
        t_crit_right = np.quantile(t_st_values, 1-alpha/2)
        accept = True if t_crit_left <= t_obs <= t_crit_right else False
        p_value = sum(t_st_values <= t_obs) / len(t_st_values) if t_obs < 0 else sum(t_st_values >= t_obs) / len(t_st_values)
    
    return accept, p_value